In [25]:
# ============================================
# 🔥 GNews + News Classifier Integration Script
# ============================================

import requests
import pandas as pd
import joblib
import numpy as np
import scipy.sparse as sp

# --- 1️⃣ Load Trained Model Bundle ---
bundle = joblib.load("/content/news_classifier_model.pkl")
model, mlb, vectorizer = bundle[0], bundle[1], bundle[2]
print("✅ Model bundle loaded successfully!")

# --- 2️⃣ Fetch News from GNews.io ---
API_KEY = "172285a1b5b6f981c517e59461b31a9a"   # 🔑 Replace with your API key
query = input("Enter a news topic (e.g., 'technology', 'sports', 'finance'): ").strip()
url = f"https://gnews.io/api/v4/search?q={query}&lang=en&country=us&max=10&apikey={API_KEY}"

response = requests.get(url)
data = response.json()

# --- 3️⃣ Parse JSON Articles into DataFrame ---
articles = []
for item in data.get("articles", []):
    title = item.get("title", "")
    description = item.get("description", "")
    content = item.get("content", "")
    text = f"{title}. {description}. {content}".strip()
    if text:
        articles.append({
            "title": title,
            "description": description,
            "content": content,
            "text": text
        })

df_articles = pd.DataFrame(articles)
print(f"✅ Retrieved {len(df_articles)} articles for query: {query}")

if df_articles.empty:
    raise ValueError("⚠️ No articles found. Try another query or check your API key.")

# --- 4️⃣ Transform Text using Saved Vectorizer ---
X = vectorizer.transform(df_articles["text"])

# --- 5️⃣ Auto-Fix Feature Mismatch (Padding/Trimming) ---
expected_features = model.n_features_in_
current_features = X.shape[1]

if current_features < expected_features:
    diff = expected_features - current_features
    print(f"⚙️ Padding {diff} missing features...")
    X = sp.hstack([X, sp.csr_matrix((X.shape[0], diff))])

elif current_features > expected_features:
    diff = current_features - expected_features
    print(f"⚙️ Trimming {diff} extra features...")
    X = X[:, :expected_features]

print(f"✅ Feature alignment done: {X.shape[1]} features match model input.")

# --- 6️⃣ Predict Labels ---
try:
    probs_list = [est.predict_proba(X)[:, 1] for est in model.estimators_]
    probs = np.column_stack(probs_list)
except AttributeError:
    probs = model.predict_proba(X)

threshold = 0.3
pred_labels = mlb.inverse_transform(probs > threshold)

# --- 7️⃣ Append Predictions ---
df_articles["predicted_labels"] = pred_labels
df_articles["top_k_labels"] = [
    [mlb.classes_[i] for i in np.argsort(p)[::-1][:3]]
    for p in probs
]

# --- 8️⃣ Display Results ---
for _, row in df_articles.iterrows():
    print("\n─────────────────────────────────────────────")
    print(f"📰 Title: {row['title']}")
    print(f"💬 Description: {row['description'][:100]}...")
    print(f"🔖 Predicted Labels: {row['predicted_labels']}")
    print(f"⭐ Top 3 Labels: {row['top_k_labels']}")

# --- 9️⃣ (Optional) Save Results ---
df_articles.to_csv("classified_news_results.csv", index=False)
print("\n📁 Results saved to classified_news_results.csv")


✅ Model bundle loaded successfully!
Enter a news topic (e.g., 'technology', 'sports', 'finance'): sports
✅ Retrieved 10 articles for query: sports
⚙️ Padding 2 missing features...
✅ Feature alignment done: 30002 features match model input.

─────────────────────────────────────────────
📰 Title: Arlington Asian American Native Hawaiian and Pacific Islander Month
💬 Description: WJLA is the local ABC affiliate for the greater Washington DC area. From our studios in Arlington, V...
🔖 Predicted Labels: ('autos', 'entertainment', 'finance', 'foodanddrink', 'health', 'lifestyle', 'movies', 'music', 'news', 'sports', 'travel', 'tv', 'video', 'weather')
⭐ Top 3 Labels: ['weather', 'travel', 'video']

─────────────────────────────────────────────
📰 Title: Sports on TV for Tuesday, Oct. 28
💬 Description: Here's what to watch on Tuesday, Oct. 28....
🔖 Predicted Labels: ('entertainment', 'finance', 'foodanddrink', 'health', 'lifestyle', 'movies', 'music', 'news', 'sports', 'travel', 'tv', 'video', 